In [1]:
#-*- coding:utf8-*-
%matplotlib inline

from __future__ import absolute_import, division, print_function, unicode_literals

from IPython.display import Image

import numpy as np

import uncertainties as unc
from uncertainties import unumpy as unp
from uncertainties.unumpy import std_devs as stds #TODO
from uncertainties.unumpy import nominal_values as noms #TODO

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D 

import seaborn as sns
sns.set(context="paper",font_scale=1.4)

import scipy as sp
import scipy.constants as const

import sympy as sym
sym.init_printing()

import pandas as pd

import ROOT as rt

import root_numpy as rnp

from aputils.latextables.tables import Table


# Fehlerfortpflanzung beim Mitteln
umean = unc.wrap(np.mean)

#Laden des Latex fonts 
plt.rcParams['text.latex.preamble']=["\\usepackage{lmodern}"]
#Options
params = {'text.usetex' : True,
          'font.size' : 11,
          'font.family' : 'lmodern',
          'text.latex.unicode': True,
          }
plt.rcParams.update(params) 

#Größe der Achsenbeschriftung
axis_font = {'size':'14'}

# Pfade
path_graphics = "../Grafiken/"
path_table = "../Tabellen/"
path_data = "../Messdaten/"

In [2]:
# Funktionen
# Achsenbeschriftungen
def axis_label(label,sym=None,unit=None):
    if sym == None:
        return lable
    elif unit == None:
        return r"".join([label," ${s}$".format(s=sym)])
    else:
        return r"".join([label," ${s}/\mathrm{{{u}}}$".format(s=sym,u=unit)])
    
# Fit Funktionen
def fit_errors(popt,pcov):
    errors = np.sqrt(np.diag(pcov))
    param_err = unp.uarray(popt,errors)
    return param_err
    
def print_errors(array,description=None):
    if description and len(description)==len(array):
        maxlen = max(map(len,description))
        lens = map(len,description)
        for i,entry in enumerate(array):
            print(description[i]," "*(maxlen-lens[i]),entry.nominal_value,"+/-",entry.std_dev)
    else:
        for i,entry in enumerate(array):
            print(entry.nominal_value,"+/-",entry.std_dev)


#Vorbereitungsaufgaben

##Aufgabe 5

In [89]:
#W2 = sym.sqrt(2)
#A_np = np.matrix([[1,1,1,0,0,0,0,0,0],[0,0,0,1,1,1,0,0,0],[0,0,0,0,0,0,1,1,1],
#               [0,0,W2,0,0,0,0,0,0],[0,W2,0,0,0,W2,0,0,0],[W2,0,0,0,W2,0,0,0,W2],
#               [0,0,0,W2,0,0,0,W2,0],[0,0,0,0,0,0,W2,0,0],[0,0,W2,0,0,W2,0,0,W2],
#               [0,W2,0,0,W2,0,0,W2,0],[W2,0,0,W2,0,0,W2,0,0]])
#A = sym.Matrix(A_np)
#B = (A.transpose()*A)
#B.inv()
# B ist singulär

In [27]:
W2 = sym.sqrt(2)
A_np = np.matrix([[0,W2,0,W2,0,0,0,0,0],[0,0,W2,0,W2,0,W2,0,0],[0,0,0,0,0,W2,0,W2,0],
                [1,1,1,0,0,0,0,0,0],[0,0,0,1,1,1,0,0,0],[0,0,0,0,0,0,1,1,1],
               [0,W2,0,0,0,W2,0,0,0],[W2,0,0,0,W2,0,0,0,W2],[0,0,0,W2,0,0,0,W2,0],
               [0,0,W2,0,0,W2,0,0,W2],[0,W2,0,0,W2,0,0,W2,0],[W2,0,0,W2,0,0,W2,0,0]])
B_np = (A_np.T * A).inv()


A = sym.Matrix(A_np)
B = (A.transpose()*A).inv()
B_np

sympy.matrices.dense.MutableDenseMatrix

In [45]:
diag = np.diag(B)
diag

array([833/2392, 493/2392, 833/2392, 511/2392, 63/299, 511/2392, 833/2392,
       493/2392, 833/2392], dtype=object)

In [46]:
var = np.array([np.float128(elem) for elem in diag])
var

array([ 0.34824415,  0.20610368,  0.34824415,  0.21362876,  0.21070234,
        0.21362876,  0.34824415,  0.20610368,  0.34824415], dtype=float128)

In [43]:
N = 1000
MU = B_np*A_np.T*sym.Matrix([N]*12)
mu = []
for m in MU:
    mu.append(float(m))
mu = np.array(mu)

In [44]:
mu

array([ 244.91541047,  336.08311944,  244.91541047,  342.43594531,
        179.159005  ,  342.43594531,  244.91541047,  336.08311944,
        244.91541047])

In [56]:
rel_error = lambda N: np.sqrt(N)/N

In [68]:
rel_error(1200)

In [70]:
35/1200